# Quake

By Cascade Tuholske, June 2020

Notebook finds zonal stats of populations for 2015 Nepal earthquakes by raster. <br><br>
**NOTE** CRS should be epsg:4326 for everything!

In [ ]:
#### Dependencies
import numpy as np
import pandas as pd
import rasterio
import geopandas as gpd
from rasterstats import zonal_stats, gen_zonal_stats
from glob import glob
import matplotlib.pyplot as plt

In [ ]:
#### Functions
def zone_loop(polys_in, rst_list, stats_type, col, split):
    """ Function loops through rasters, calcs zonal_stats and returns stats as a data frame.
    Args:
        polys_in = polygons
        rst_list = list of paths & fns of rasters
        stats_type = stats type for each poly gone (see zonal stats)
        col = column to merge it all
        split = where to split the file name string (e.g. _matched.tif)
    """
    
    # copy polys to write out
    polys_out = polys_in.copy()
    
    for rst in rst_list:
        
        # Get data name
        data = rst.split(DATA_PATH+'interim/')[1].split(split)[0]
        print('Started', data)
        
        # Run zonal stats
        zs_feats = zonal_stats(polys_in, rst, stats=stats_type, geojson_out=True)
        zgdf = gpd.GeoDataFrame.from_features(zs_feats, crs=polys_in.crs)
        
        # Rename columns and merge
        zgdf = zgdf.rename(columns={stats_type: data+'_'+stats_type})
        
        polys_out = polys_out.merge(zgdf[[col, data+'_'+stats_type]], on = col, how = 'inner')
    
    return polys_out

In [ ]:
def poly_prep(polys_fn, col):
    "function opens earth quake polygons for zonal loop"
    
    # open
    polys = gpd.read_file(polys_fn)
    
    # subset, be sure to check the admin level
    polys = polys[['geometry', col]]
    
    return polys

## Run on Quake Intensity Contours (MI) from USGS for Nepal 2015



In [ ]:
# Git tif files
rst_fns = glob('/Users/cascade/Github/PopGridCompare/data/interim/*all_Nepal.tif')
rst_fns

In [ ]:
# open polys
DATA_PATH = '/Users/cascade/Github/PopGridCompare/data/'
nepal_polys_fn = DATA_PATH+'raw/USGS_Data/Nepal/shape/mi.shp'

col = 'PARAMVALUE'
nepal_polys = poly_prep(nepal_polys_fn, col)

In [ ]:
# Run zonal stats loop
nepal_polys_sum = zone_loop(nepal_polys, rst_fns, 'sum', col, '_all_Nepal.tif')

In [ ]:
#### Save the poly sums
nepal_polys_sum.to_file(DATA_PATH+'/interim/nepal_quake_pop.shp')

In [ ]:
#### plot it Nepal
data = nepal_polys_sum[nepal_polys_sum['PARAMVALUE'] > 6]
title = '2015 Nepal Earth Quake > 6 Intensity'

fig, axs = plt.subplots(1, 1, figsize = (8, 8), sharex=True)
ws = 0.25
fig.subplots_adjust(wspace=ws)

axs.scatter(data['PARAMVALUE'], data['GPWv4_sum'], color = 'black', alpha = 0.5)
axs.scatter(data['PARAMVALUE'], data['GHS15_sum'], color = 'red', alpha = 0.5)
axs.scatter(data['PARAMVALUE'], data['LS15_sum'], color = 'orange', alpha = 0.5)
axs.scatter(data['PARAMVALUE'], data['ESRI16_sum'], color = 'blue', alpha = 0.5)
axs.scatter(data['PARAMVALUE'], data['WP16_sum'], color = 'green', alpha = 0.5)

axs.set_title(title, size = 15 )
axs.set_xlabel('Quake Intensity', fontsize = 13)
axs.set_ylabel('Total Population', fontsize = 13)
#plt.ylim([100,10000000])

plt.yscale('linear')
axs.ticklabel_format(useOffset=False, style = 'plain')

labels = ['GPWv4-15','GHSL-15', 'LS-15', 'WPE-16', 'WP-16']
plt.legend(labels)

# save it out
fig_out = '/Users/cascade/Github/PopGridCompare/data/FIGS/AGU2020/Nepal2015_all.png'
plt.savefig(fig_out, dpi = 300, facecolor = 'white', bbox_inches='tight')

# Urban

In [ ]:
# Git tif files
rst_fns = glob('/Users/cascade/Github/PopGridCompare/data/interim/*_urban_Nepal.tif')
rst_fns

In [ ]:
# open polys
DATA_PATH = '/Users/cascade/Github/PopGridCompare/data/'
nepal_polys_fn = DATA_PATH+'raw/USGS_Data/Nepal/shape/mi.shp'

col = 'PARAMVALUE'
nepal_polys = poly_prep(nepal_polys_fn, col)


In [ ]:
# Run zonal stats loop
nepal_polys_sum = zone_loop(nepal_polys, rst_fns, 'sum', col, '_urban_Nepal.tif')


In [ ]:
#### Save the poly sums
nepal_polys_sum.to_file(DATA_PATH+'/interim/nepal_urban_quake_pop.shp')

In [ ]:
#### plot it Nepal
data = nepal_polys_sum[nepal_polys_sum['PARAMVALUE'] > 6]
title = 'Urban 2015 Nepal Earth Quake > 6 Intensity'

fig, axs = plt.subplots(1, 1, figsize = (8, 8), sharex=True)
ws = 0.25
fig.subplots_adjust(wspace=ws)

axs.scatter(data['PARAMVALUE'], data['GPWv4_sum'], color = 'black', alpha = 0.5)
axs.scatter(data['PARAMVALUE'], data['GHS15_sum'], color = 'red', alpha = 0.5)
axs.scatter(data['PARAMVALUE'], data['LS15_sum'], color = 'orange', alpha = 0.5)
axs.scatter(data['PARAMVALUE'], data['ESRI16_sum'], color = 'blue', alpha = 0.5)
axs.scatter(data['PARAMVALUE'], data['WP16_sum'], color = 'green', alpha = 0.5)

axs.set_title(title, size = 15 )
axs.set_xlabel('Quake Intensity', fontsize = 13)
axs.set_ylabel('Total Population', fontsize = 13)
#plt.ylim([100,10000000])

plt.yscale('linear')
axs.ticklabel_format(useOffset=False, style = 'plain')

labels = ['GPWv4-15','GHSL-15', 'LS-15', 'ESRI-16', 'WP-16']
plt.legend(labels)

# save it out
fig_out = '/Users/cascade/Github/PopGridCompare/data/FIGS/AGU2020/Nepal2015_urban.png'
plt.savefig(fig_out, dpi = 300, facecolor = 'white', bbox_inches='tight')

# Rural

In [ ]:
# Git tif files
rst_fns = glob('/Users/cascade/Github/PopGridCompare/data/interim/*_rural_Nepal.tif')
rst_fns

In [ ]:
# open polys
DATA_PATH = '/Users/cascade/Github/PopGridCompare/data/'
nepal_polys_fn = DATA_PATH+'raw/USGS_Data/Nepal/shape/mi.shp'

col = 'PARAMVALUE'
nepal_polys = poly_prep(nepal_polys_fn, col)


In [ ]:
# Run zonal stats loop
nepal_polys_sum = zone_loop(nepal_polys, rst_fns, 'sum', col, '_rural_Nepal.tif')


In [ ]:
#### Save the poly sums
nepal_polys_sum.to_file(DATA_PATH+'/interim/nepal_rural_quake_pop.shp')

In [ ]:
#### plot it Nepal
data = nepal_polys_sum[nepal_polys_sum['PARAMVALUE'] > 6]
title = 'Rural 2015 Nepal Earth Quake > 6 Intensity'

fig, axs = plt.subplots(1, 1, figsize = (8, 8), sharex=True)
ws = 0.25
fig.subplots_adjust(wspace=ws)

axs.scatter(data['PARAMVALUE'], data['GPWv4_sum'], color = 'black', alpha = 0.5)
axs.scatter(data['PARAMVALUE'], data['GHS15_sum'], color = 'red', alpha = 0.5)
axs.scatter(data['PARAMVALUE'], data['LS15_sum'], color = 'orange', alpha = 0.5)
axs.scatter(data['PARAMVALUE'], data['ESRI16_sum'], color = 'blue', alpha = 0.5)
axs.scatter(data['PARAMVALUE'], data['WP16_sum'], color = 'green', alpha = 0.5)

axs.set_title(title, size = 15 )
axs.set_xlabel('Quake Intensity', fontsize = 13)
axs.set_ylabel('Total Population', fontsize = 13)
#plt.ylim([100,10000000])

plt.yscale('linear')
axs.ticklabel_format(useOffset=False, style = 'plain')

labels = ['GPWv4-15','GHSL-15', 'LS-15', 'ESRI-16', 'WP-16']
plt.legend(labels)

# save it out
fig_out = '/Users/cascade/Github/PopGridCompare/data/FIGS/AGU2020/Nepal2015_rural.png'
plt.savefig(fig_out, dpi = 300, facecolor = 'white', bbox_inches='tight')

## Analysis

In [ ]:
# All
all_pop = gpd.read_file(DATA_PATH+'/interim/nepal_quake_pop.shp')
all_pop['max'] = all_pop.iloc[:,1:5].values.max(1)
all_pop['min'] = all_pop.iloc[:,1:5].values.min(1)
all_pop['dif'] = all_pop['max'] - all_pop['min']

In [ ]:
# Urban 
urban_pop = gpd.read_file(DATA_PATH+'/interim/nepal_urban_quake_pop.shp')
urban_pop['max'] = urban_pop.iloc[:,1:5].values.max(1)
urban_pop['min'] = urban_pop.iloc[:,1:5].values.min(1)
urban_pop['dif'] = urban_pop['max'] - urban_pop['min']

In [ ]:
df = pd.DataFrame()
df['PARAMVALUE'] = urban_pop['PARAMVALUE']
df['dif'] = urban_pop['WP16_sum'] - urban_pop['GHS15_sum']
df;

In [ ]:
# rural 
rural_pop = gpd.read_file(DATA_PATH+'/interim/nepal_rural_quake_pop.shp')
rural_pop['max'] = rural_pop.iloc[:,1:5].values.max(1)
rural_pop['min'] = rural_pop.iloc[:,1:5].values.min(1)
rural_pop['dif'] = rural_pop['max'] - rural_pop['min']

In [ ]:
rural_pop